In [11]:
import numpy as np
from scipy import signal

In [12]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    # вычисляет выходной сигнал Y слоя для заданного входного сигнала X
    def forward_propagation(self, input):
        raise NotImplementedError

    # вычисляет dE/dX для заданного dE/dY (и обновляет параметры)
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

In [13]:
class FCLayer(Layer):
    # input_size = число входных нейронов
    # output_size = число output нейронов
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size)
        self.bias = np.random.rand(1, output_size)

    # returns output for a given input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    # считаем dE/dW, dE/dB чтобы получить output_error=dE/dY. Возвращаем input_error=dE/dX.
    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T) #dE/dx
        weights_error = np.dot(self.input.T, output_error) #dE/dw
        # dBias = output_error

        # обновляем параметры
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

In [14]:
class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    # возвращаем activated input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    # input_error=dE/dX, output_error=dE/dY.
    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

In [15]:
class ConvLayer(Layer):
    # input_shape = (i,j,d)
    # kernel_shape = (m,n)
    # layer_depth = output_depth
    def __init__(self, input_shape, kernel_shape, layer_depth):
        self.input_shape = input_shape
        self.input_depth = input_shape[2]
        self.kernel_shape = kernel_shape
        self.layer_depth = layer_depth
        self.output_shape = (input_shape[0]-kernel_shape[0]+1, input_shape[1]-kernel_shape[1]+1, layer_depth)
        self.weights = np.random.rand(kernel_shape[0], kernel_shape[1], self.input_depth, layer_depth)
        self.bias = np.random.rand(layer_depth)

    def forward_propagation(self, input):
        self.input = input
        self.output = np.zeros(self.output_shape)

        for k in range(self.layer_depth):
            for d in range(self.input_depth):
                self.output[:,:,k] += signal.correlate2d(self.input[:,:,d], self.weights[:,:,d,k], 'valid') + self.bias[k]

        return self.output

    # считаем dE/dW, dE/dB чтобы получть output_error=dE/dY. Возвращаем input_error=dE/dX.
    def backward_propagation(self, output_error, learning_rate):
        in_error = np.zeros(self.input_shape)
        dWeights = np.zeros((self.kernel_shape[0], self.kernel_shape[1], self.input_depth, self.layer_depth))
        dBias = np.zeros(self.layer_depth)

        for k in range(self.layer_depth):
            for d in range(self.input_depth):
                in_error[:,:,d] += signal.convolve2d(output_error[:,:,k], self.weights[:,:,d,k], 'full')
                dWeights[:,:,d,k] = signal.correlate2d(self.input[:,:,d], output_error[:,:,k], 'valid')
            dBias[k] = self.layer_depth * np.sum(output_error[:,:,k])

        self.weights -= learning_rate*dWeights
        self.bias -= learning_rate*dBias
        return in_error

In [16]:
class FlattenLayer(Layer):
    # returns the flattened input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = input_data.flatten().reshape((1,-1))
        return self.output

    # Returns input_error=dE/dX for a given output_error=dE/dY.
    # learning_rate is not used because there is no "learnable" parameters.
    def backward_propagation(self, output_error, learning_rate):
        return output_error.reshape(self.input.shape)

In [17]:
#loss_func
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/tf.size(y_true);

def BinaryCrossEntropy(y_true, y_pred):
    return -(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred)).mean()
    
def BinaryCrossEntropy_prime(y_true, y_pred):  
    return (-y_true/y_pred) + (1 - y_true)/(1 - y_pred)
    

def _softmax(X):
    exps = np.exp(X)
    return exps / np.sum(exps)

#activ_func
def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1 - np.tanh(x)**2

def relu(x):
    return np.maximum(x, 0)

def relu_prime(x):
    return np.array(x >= 0).astype('int')

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x) * (1 - sigmoid(x))

In [18]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    def add(self, layer):
        self.layers.append(layer)

    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    def predict(self, input_data):
        samples = len(input_data)
        result = []

        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    def fit(self, x_train, y_train, epochs, learning_rate):
        samples = len(x_train)

        for i in range(epochs):
            err = 0
            for j in range(samples):
                
                # forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)
                    
                # backward propagation
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)

In [19]:
# from sklearn.model_selection import train_test_split
# from sklearn.datasets import make_regression, make_classification
# from sklearn.metrics import classification_report, confusion_matrix

In [20]:
# x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
# y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

# net = Network()
# net.add(FCLayer(2, 3))
# net.add(ActivationLayer(tanh, tanh_prime))
# net.add(FCLayer(3, 1))
# net.add(ActivationLayer(tanh, tanh_prime))

# # train
# net.use(mse, mse_prime)
# net.fit(x_train, y_train, epochs=1000, learning_rate=0.1)

# # test
# out = net.predict(x_train)
# print(out)

In [21]:
# X, y = make_classification(n_samples=100, n_features=10, random_state=42)
# X = X.reshape(X.shape[0], 1, X.shape[1])
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# print(y_train.shape)
# input_shape = X_train.shape[2]

# net = Network()
# net.add(FCLayer(input_shape, 2))
# net.add(ActivationLayer(tanh, tanh_prime))
# net.add(FCLayer(2, 1))
# net.add(ActivationLayer(tanh, tanh_prime))

# net.use(mse, mse_prime)
# net.fit(X_train, y_train, epochs=10, learning_rate=0.05)

# predictions = net.predict(X_test)
# predictions = np.array(predictions)
# nsamples, nx, ny = predictions.shape
# predictions = predictions.reshape((nsamples,nx*ny))
# predictions = np.around(predictions)

# print(classification_report(y_test, predictions))

In [22]:
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler, MinMaxScaler
# import pandas as pd
# from math import sqrt
# import numpy as np

# df_classification = pd.read_csv('D:/ML/data/classification/airlines_task_preprocessed.csv', index_col=0)
# y_cf = df_classification[["Delay"]]
# X_cf = df_classification.drop(["Delay"], axis=1)

# mms = MinMaxScaler()
# mms.fit(X_cf)
# X_cf = mms.transform(X_cf)
# X = X_cf
# X = X.reshape(X.shape[0], 1, X.shape[1])
# X_train, X_test, y_train, y_test = train_test_split(X, y_cf, test_size=0.2, random_state=4)
# input_shape = X_train.shape[2]
# y_train = np.array(y_train).flatten()
# print(y_train.shape)
# net = Network()
# net.add(FCLayer(input_shape, 2))
# net.add(ActivationLayer(tanh, tanh_prime))
# net.add(FCLayer(2, 1))
# net.add(ActivationLayer(tanh, tanh_prime))

# net.use(mse, mse_prime)
# net.fit(X_train, y_train, epochs=10, learning_rate=0.05)

# predictions = net.predict(X_test)
# predictions = np.array(predictions)
# nsamples, nx, ny = predictions.shape
# predictions = predictions.reshape((nsamples,nx*ny))
# predictions = np.around(predictions)

# print(classification_report(y_test, predictions))

In [23]:
IMG_WIDTH=200
IMG_HEIGHT=200
img_folder= "D:/ML/data/data3"

In [24]:
import os
from PIL import Image
import tensorflow as tf
def create_dataset_PIL(img_folder):
    
    img_data_array=[]
    class_name=[]
    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):
       
            image_path= os.path.join(img_folder, dir1,  file)
            image= np.array(Image.open(image_path))
            image= np.resize(image,(IMG_HEIGHT,IMG_WIDTH,3))
            image = image.astype('float32')
            image /= 255  
            img_data_array.append(image)
            class_name.append(dir1)
    return img_data_array , class_name  

In [25]:
PIL_img_data, class_name=create_dataset_PIL(img_folder)
target_dict={k: v for v, k in enumerate(np.unique(class_name))}
target_val=  [target_dict[class_name[i]] for i in range(len(class_name))]

In [28]:
net = Network()
net.add(ConvLayer((200,200,3), (3,3), 1))
net.add(ActivationLayer(relu, relu_prime))
net.add(FlattenLayer())
net.add(FCLayer(39204, 1))
net.add(ActivationLayer(sigmoid, sigmoid_prime))
net.use(mse, mse_prime)
net.fit(np.array(PIL_img_data[::1000]), tf.cast(list(map(int,target_val[::1000])),tf.int32), epochs=50, learning_rate=0.1)

In [33]:
img = tf.keras.preprocessing.image.load_img(
    "D:/ML/data/data3/cats/9.jpg", target_size=(200, 200)
)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = net.predict(img_array)
predictions

[array([[1.]])]